In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.signal import savgol_filter

# Load data
filename = '../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
loaded_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert 'timestamp' to datetime and set it as the index
loaded_df['timestamp'] = pd.to_datetime(loaded_df['timestamp'])
loaded_df.set_index('timestamp', inplace=True)

# Convert all price and volume columns to numeric format
for column in ['open', 'high', 'low', 'close', 'volume']:
    loaded_df[column] = pd.to_numeric(loaded_df[column], errors='coerce')

In [ ]:
loaded_df.shape

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'loaded_df' is already defined and contains your data with a datetime index
df = loaded_df[:300]

# Set the window sizes for the moving averages
window_short = 10  # Example: short-term window (e.g., 10 minutes)
window_long = 30   # Example: long-term window (e.g., 30 minutes)
window_xl = 90

# window_short = window_short + 1 if window_short / 2 == 0 else window_short
# window_long = window_long + 1 if window_long / 2 == 0 else window_long

# Calculate the short-term moving average using a rolling window
#ma_short = df['close'].rolling(window=window_short).mean()
polyorder = 3       # Степень полинома
delta = 1           # Временной шаг между точками (замените на реальное значение, если нужно)
ma_short = savgol_filter(df['close'].values, window_length=window_short, 
                         polyorder=polyorder, deriv=0, delta=delta)

# Calculate the long-term moving average using a rolling window
#ma_long = df['close'].rolling(window=window_long).mean()
ma_long = savgol_filter(df['close'].values, window_length=window_long,
                            polyorder=polyorder, deriv=0, delta=delta)

ma_xl = savgol_filter(df['close'].values, window_length=window_xl,
                            polyorder=polyorder, deriv=0, delta=delta)

# Compute the difference between the short-term and long-term moving averages
ma_diff = ma_short - ma_long
xl_diff = ma_long - ma_xl
ma_diff_sg = savgol_filter(ma_diff, window_length=window_short,
                            polyorder=polyorder, deriv=0, delta=delta)
ma_diff_sg_prime = savgol_filter(ma_diff, window_length=window_short,
                            polyorder=polyorder, deriv=1, delta=delta)
ma_diff_sg_prime2 = savgol_filter(ma_diff, window_length=window_short,
                            polyorder=polyorder, deriv=2, delta=delta)
xl_diff_sg = savgol_filter(xl_diff, window_length=window_long,
                            polyorder=polyorder, deriv=0, delta=delta)
xl_diff_sg_prime = savgol_filter(xl_diff, window_length=window_long,
                            polyorder=polyorder, deriv=1, delta=delta)


In [ ]:
# Plot the close price and the two moving averages
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['close'], label='Close Price')
plt.plot(df.index, ma_short, label=f'MA Short ({window_short})')
plt.plot(df.index, ma_long, label=f'MA Long ({window_long})')
plt.title('Price and Moving Averages')
plt.legend()
plt.show()

# Plot the difference between the moving averages
plt.figure(figsize=(14, 6))
#plt.plot(df.index, ma_diff, label='Short - Long')
plt.plot(df.index, ma_diff_sg, label='sg(Short - Long)')
plt.legend()
plt.show()
# Plot the difference between the moving averages
plt.figure(figsize=(14, 6))
plt.plot(df.index, ma_diff, label='Short - Long')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(ma_diff_sg, ma_diff_sg_prime, label='fp')
plt.legend()
plt.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Create a Plotly figure
fig = go.Figure()

# Add a scatter plot
fig.add_trace(go.Scatter(
    x=ma_diff_sg,
    y=ma_diff_sg_prime,
    #mode='lines+markers',
    mode='markers',
    marker=dict( size=1,),
    name='fp'
))

# Update layout for better visualization
fig.update_layout(
    title='Interactive Plot with Zoom and Pan',
    xaxis_title='ma_diff_sg',
    yaxis_title='ma_diff_sg_prime',
    width=600,
    height=600
)

# Show the plot
fig.show()

In [ ]:
x = ma_diff_sg
y = ma_diff_sg_prime
#z = ma_diff_sg_prime2
z = xl_diff_sg_prime

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(ma_diff_sg, xl_diff_sg_prime, label='fp')
plt.legend()
plt.show()

In [ ]:
cnt = 0
fx = []
fy = []
fz = []
appending = 0
xPos = 8
xDelta = 0.5
yPos = -0.5
yDelta = 0.4
zPos = 0
zDelta = 10
for i in range(0, len(x)):
    if appending == 0 and xPos < x[i] < xPos + xDelta and yPos < y[i] < yPos + yDelta and zPos < z[i] < zPos + zDelta:
        cnt += 1
        appending = 50
        fx.append(None)
        fy.append(None)
        fz.append(None)
        fx.append(x[i])
        fy.append(y[i])
        fz.append(z[i])
        continue
    if appending > 0:
        appending -= 1
        # if y[i] < 0 and y[i-1] > 0:
        #     appending = 0
        #     continue
        fx.append(x[i])
        fy.append(y[i])
        fz.append(z[i])
print(cnt)
for i in range(0, len(fx)):
    if (fx[i] == None):
        print(i)

In [ ]:
import plotly.graph_objs as go

fig = go.Figure(data=[go.Scatter3d(x=fx, y=fy, z=fz, mode='lines')])
fig.update_layout(width=800, height=600, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Create a Plotly figure
fig = go.Figure()

# Add a scatter plot
fig.add_trace(go.Scatter(
    x=fx,
    y=fy,
    mode='lines',#mode='lines+markers',
    name='fp'
))

# Update layout for better visualization
fig.update_layout(
    title='Interactive Plot with Zoom and Pan',
    xaxis_title='ma_diff_sg',
    yaxis_title='ma_diff_sg_prime',
    width=600,
    height=600
)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objs as go

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x[:1000],
    y=y[:1000],
    z=z[:1000],
    mode='lines',
    #line_shape='spline'
    line=dict(
        color='blue',
        width=2
    ),
)])

# Update layout for better visualization
fig.update_layout(
    width=800,
    height=600,
    margin=dict(l=10, r=10, b=10, t=10),
    #line_shape='spline'
)

# Show the plot
fig.show()

In [ ]:
def trajectory_to_image(x, y, img_size=100, light_gray=200, color_pow=40):
    """
    Принимает координаты траектории (x, y) и создает изображение размером img_size x img_size,
    где:
      - пиксели, в которые не попадала траектория, белые (255);
      - если в пиксель попала точка ровно 1 раз, его серый цвет равен light_gray (например, 200);
      - пиксели с максимальным числом попаданий становятся чёрными (0);
      - для остальных используется линейный градиент между light_gray и 0.
    """
    # Определяем диапазоны координат
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)
    
    # Инициализируем матрицу подсчёта попаданий (плотности) размером img_size x img_size
    density_matrix = np.zeros((img_size, img_size), dtype=np.int32)
    
    # Проходим по всем точкам траектории и определяем, в какой пиксель попадает каждая точка
    for xi, yi in zip(x, y):
        # Преобразуем координаты в индексы от 0 до img_size-1
        # Используем int() (это аналог np.floor, если число положительное)
        ix = int((xi - x_min) / (x_max - x_min) * (img_size - 1))
        iy = int((yi - y_min) / (y_max - y_min) * (img_size - 1))
        density_matrix[iy, ix] += 1

    # Определяем максимальное число попаданий (игнорируя ячейки с 0)
    if np.any(density_matrix > 0):
        max_count = density_matrix[density_matrix > 0].max()
    else:
        max_count = 1  # на случай, если ни одна точка не попала (не должно случаться)

    # Создаем итоговое изображение: изначально все пиксели белые (255)
    img = np.full((img_size, img_size), 255, dtype=np.uint8)
    
    # Для ячеек, в которые попала траектория, задаем интенсивность по градиенту:
    # если попадание = 1, то интенсивность = light_gray (светло-серый),
    # если попадание = max_count, то интенсивность = 0 (чёрный);
    # для промежуточных значений выполняется линейная интерполяция.
    mask = density_matrix > 0
    if max_count == 1:
        # Если максимум равен 1, то во всех заполненных ячейках назначаем light_gray
        img[mask] = light_gray
    else:
        # Линейное преобразование: для count c (c>=1)
        # intensity = light_gray - (light_gray * (c - 1) / (max_count - 1))
        #intensities = light_gray - (light_gray * (density_matrix[mask] - 1) / max_count)
        intensities = light_gray  * pow(1 - density_matrix[mask] / max_count, color_pow)
        # Гарантируем, что значение находится в диапазоне 0...255
        intensities = np.clip(intensities, 0, 255)
        img[mask] = intensities.astype(np.uint8)
    
    return img

In [ ]:
# Получаем изображение 100x100 по траектории
img = trajectory_to_image(x, y, img_size=300, light_gray=200)

# Вычисляем extent для осей
x_min, x_max = np.min(x), np.max(x)
y_min, y_max = np.min(y), np.max(y)
extent = [x_min, x_max, y_min, y_max]

# Вычисляем соотношение сторон (aspect ratio) по данным
# Это отношение ширины диапазона по x к высоте диапазона по y
aspect_ratio = (x_max - x_min) / (y_max - y_min)

# Отображаем полученное изображение
plt.figure(figsize=(8, 8))
# Параметр origin='lower' задаёт нижнюю левую точку в качестве начала координат,
# extent позволяет масштабировать оси в соответствии с исходными координатами
plt.imshow(img, cmap='gray', origin='lower', extent=extent, aspect=aspect_ratio)
plt.xlabel("X")
plt.ylabel("Y")
plt.show()
extent

In [ ]:

# Define the zoom region boundaries
x_min_zoom, x_max_zoom = -50, 50
y_min_zoom, y_max_zoom = -2, 2

# Create a boolean mask to filter points within the zoom area
# English comment: Filter points with x between x_min_zoom and x_max_zoom and y between y_min_zoom and y_max_zoom
mask = (x >= x_min_zoom) & (x <= x_max_zoom) & (y >= y_min_zoom) & (y <= y_max_zoom)

# Apply the mask to get the filtered x and y arrays
x_zoom = x[mask]
y_zoom = y[mask]

# Вычисляем соотношение сторон (aspect ratio) по данным
# Это отношение ширины диапазона по x к высоте диапазона по y
aspect_ratio = (x_max_zoom - x_min_zoom) / (y_max_zoom - y_min_zoom)

# Generate the density image using the trajectory_to_image function for the filtered points
img_zoom = trajectory_to_image(x_zoom, y_zoom, img_size=200, light_gray=200, color_pow=4)

# Set the extent to match the zoom region
extent_zoom = [x_min_zoom, x_max_zoom, y_min_zoom, y_max_zoom]

# Plot the zoomed image
plt.figure(figsize=(6, 6))
plt.imshow(img_zoom, cmap='gray', origin='lower', extent=extent_zoom, aspect=aspect_ratio)
plt.xlabel("X")
plt.ylabel("Y")
plt.show()